In [1]:
from typing import Tuple, List, Dict, Optional, Iterator
from queue import PriorityQueue

In [3]:
GridLocation = Tuple[int, int]

In [31]:
class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: List[GridLocation] = []

    def isInBounds(self, location: GridLocation) -> bool:
        """Check if the location is within the grid bounds."""
        x, y = location
        return 0 <= x < self.width and 0 <= y < self.height

    def isPassable(self, location: GridLocation) -> bool:
        """Check if the location is passable (not a wall)."""
        return location not in self.walls

    def getNeighbors(self, location: GridLocation) -> Iterator[GridLocation]:
        """Get neighboring locations of the current location."""
        x, y = location
        neighbors = [(x + 1, y), (x - 1, y), (x, y - 1), (x, y + 1)]  # East, West, North, South
        # reverse for tie-breaking to avoid ugly paths
        if (x + y) % 2 == 0:
            neighbors.reverse()  # South, North, West, East
        # filter neighbors to keep only those in bounds and passable
        neighbors = filter(self.isInBounds, neighbors)
        neighbors = filter(self.isPassable, neighbors)
        return neighbors

    def getCost(self, fromNode: GridLocation, toNode: GridLocation) -> int:
        """Get the movement cost between two nodes."""
        return 1  # Default movement cost


In [14]:
def heuristic(a: GridLocation, b: GridLocation) -> float:
     """Calculate the Manhattan distance as the heuristic."""
     x1, y1 = a
     x2, y2 = b
     return abs(x1 - x2) + abs(y1 - y2)

In [33]:
def aStarSearch(grid: SquareGrid, start: GridLocation, goal: GridLocation):
    """Perform the A* Search algorithm."""
    frontier = PriorityQueue()  # priority queue to manage the frontier
    frontier.put((0, start))  # (priority, location)

    cameFrom: Dict[GridLocation, Optional[GridLocation]] = {}
    costSoFar: Dict[GridLocation, float] = {}

    cameFrom[start] = None
    costSoFar[start] = 0

    while not frontier.empty():
        _, current = frontier.get()

        if current == goal:
            break

        for neighbor in grid.getNeighbors(current):
            newCost = costSoFar[current] + grid.getCost(current, neighbor)
            if neighbor not in costSoFar or newCost < costSoFar[neighbor]:
                costSoFar[neighbor] = newCost
                priority = newCost + heuristic(neighbor, goal)
                frontier.put((priority, neighbor))
                cameFrom[neighbor] = current

    return cameFrom, costSoFar


In [19]:
def reconstructPath(cameFrom: Dict[GridLocation, GridLocation], 
 start: GridLocation, goal: GridLocation) -> List[GridLocation]:
     """Reconstruct the path from start to goal."""
     current = goal
     path = []
     while current != start:
         path.append(current)
         current = cameFrom[current]
         path.append(start)
         path.reverse()
     return path

In [35]:
def drawGrid(grid: SquareGrid, path: List[GridLocation] = None, directions: Dict[GridLocation, GridLocation] = None, start: GridLocation = None, goal: GridLocation = None):
    """Visualize the grid."""
    for y in range(grid.height):
        for x in range(grid.width):
            location = (x, y)
            if location == start:
                print("S", end=" ")  # start point
            elif location == goal:
                print("E", end=" ")  # end point
            elif location in grid.walls:
                print("|", end=" ")  # wall
            elif path and location in path:
                print("*", end=" ")  # path
            elif directions and location in directions:
                dx, dy = directions[location][0] - x, directions[location][1] - y
                if dx == 1:
                    print(">", end=" ")
                elif dx == -1:
                    print("<", end=" ")
                elif dy == 1:
                    print("v", end=" ")
                elif dy == -1:
                    print("^", end=" ")
            else:
                print(".", end=" ")  # Empty space
        print()



In [37]:
#set up the grid
grid = SquareGrid(10, 10)
grid.walls = [
 (1, 7), (2, 7), (3, 3), (3, 7), (4, 7), (0, 4), (5, 7), 
 (3, 6), (3, 5), (7, 2), (8, 2), (9, 2)
]

#start and goal points
start, goal = (0, 0), (9, 9)

#perform A* search
cameFrom, costSoFar = aStarSearch(grid, start, goal)

#visualize the results
print("Grid with path:")
path = reconstructPath(cameFrom, start, goal)
drawGrid(grid, path=path)

print("\nGrid with directions:")
drawGrid(grid, directions=cameFrom, start=start, goal=goal)

Grid with path:
* . . . . . . . . . 
* . . . . . . . . . 
* . . . . . . | | | 
* * . | . . . . . . 
| * * * * . . . . . 
. . . | * . . . . . 
. . . | * * * . . . 
. | | | | | * . . . 
. . . . . . * . . . 
. . . . . . * * * * 

Grid with directions:
S < < < < < < < < < 
^ < < < < < < < < < 
^ < < < < < < | | | 
^ < < | ^ < < < < < 
| ^ < < < < < < < < 
> ^ < | ^ < < < < < 
> ^ < | ^ < < < < < 
. | | | | | ^ < < < 
. . . . . > ^ < < < 
. . . . . > ^ < < E 
